In [4]:
!pip install groq

In [16]:
from google.colab import userdata
import os
import json
import time

from groq import Groq

client = Groq(api_key= userdata.get('GROQ_API_KEY'))

userdata.get('GROQ_API_KEY')

'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'

In [20]:
!pip install langchain-groq


In [72]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

final_dict = {}


with open('testing_data_set.json', 'r') as file:
    test_json = json.load(file)

# Initialize Groq LLM
llm = ChatGroq(
    model_name="deepseek-r1-distill-llama-70b",
    temperature=0.7,
    api_key=userdata.get('GROQ_API_KEY')
)

# Define the expected JSON structure
parser = JsonOutputParser(pydantic_object={
    "type": "object",
    "properties": {
        "department": {"type": "string"},
        "priority": {"type": "string"},
        "language": {"type": "string"},
        "type": {"type": "string"}
    }
})

# Create a simple prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """Classify IT support tickets into JSON with this structure:
        {{
            "department": "Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, General Inquiry",
            "priority": "low, medium, high",
            "language": "en, es, fr, de, etc. (full list at https://docs.aws.amazon.com/translate/latest/dg/what-is-languages.html#what-is-languages-supported)",
            "type": "Incident, Request, Problem, Change"
        }}

        Examples:
        [
            {{
                "ticket_ID": 1001,
                "ticket_subject": "Discrepancia de facturación en Google Workspace",
                "ticket_body": "Monto de facturación incorrecto.",
                "department": "Billing and Payments",
                "type": "Incident",
                "priority": "low",
                "language": "es"
            }},
            {{
                "ticket_ID": 1002,
                "ticket_subject": "Urgent Consultation Request for Critical IT Issues",
                "ticket_body": "Critical server issues.",
                "department": "Customer Service",
                "type": "Request",
                "priority": "high",
                "language": "en"
            }},
            {{
                "ticket_ID": 1003,
                "ticket_subject": "Consulta sobre Servicios de Consultoría en TI",
                "ticket_body": "Info on IT consulting.",
                "department": "General Inquiry",
                "type": "Request",
                "priority": "medium",
                "language": "es"
            }},
            {{
                "ticket_ID": 1004,
                "ticket_subject": "Demande de mise à jour des dossiers",
                "ticket_body": "Update employee records.",
                "department": "Human Resources",
                "type": "Change",
                "priority": "low",
                "language": "fr"
            }},
            {{
                "ticket_ID": 1005,
                "ticket_subject": "Issues with Slack connection affecting team communication today",
                "ticket_body": "Problems with Slack.",
                "department": "Product Support",
                "type": "Problem",
                "priority": "medium",
                "language": "en"
            }}
        ]
    """),
    ("user", "{input}")
])

# Create the chain that guarantees JSON output
chain = prompt | llm | parser

def classify_ticket(ticket: dict) -> dict:
    result = chain.invoke({"input": json.dumps(ticket)})
    print(json.dumps(result, indent=2))
    final_dict[ticket['ticket_ID']] = [result.get("department"), result.get("type"), result.get("priority"), result.get("language")]

for i in range(0, 500):
    print(test_json[i])
    classify_ticket(test_json[i])
    time.sleep(10)

{'ticket_ID': 1, 'ticket_subject': 'Urgente: problema con la funzione di condivisione dello schermo di Zoom', 'ticket_body': "Gentile assistenza clienti,\n\nSto riscontrando un incidente di alta priorità in cui la condivisione dello schermo non riesce durante le sessioni di webinar aziendali critiche utilizzando Zoom versione 5.11.0. Questo problema sta interrompendo le nostre operazioni e richiede un'attenzione immediata.\n\nPotresti risolvere il problema il prima possibile o indirizzarmi a qualcuno che può aiutarti?\n\nGrazie per il tuo supporto urgente.\n\nCordiali saluti,\n\n<name>\n<acc_num>"}


KeyboardInterrupt: 

In [73]:
final_dict

{}

In [70]:
import csv

# Define the header
header = ['ticket_ID', 'department', 'type', 'priority', 'language']

# Write data to CSV file
with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for ticket_id, values in final_dict.items():
        writer.writerow([ticket_id] + values)

print("Data has been written to submission.csv")

Data has been written to submission.csv
